In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
pages = {
    'QB': 5,
    'RB': 11,
    'WR': 16,
    'TE': 10,
}
weeks = range(1, 18)

In [59]:
def cbs(pos, week):
    url = 'https://football.fantasysports.yahoo.com/f1/731507/players?&sort=AR&sdir=1&status=ALL&pos={pos}&stat1=S_PW_{week}&jsenabled=1&count={i}'.format(pos=pos, week=week, i="{i}")
    try:
        driver.get(url.format(i=0))
        time.sleep(3)
        elem = driver.find_element_by_class_name("Table")
        soup = BeautifulSoup(elem.get_attribute("innerHTML"), "html.parser")
        pheader = soup.find('thead').find_all('tr')[1].find_all('th')
        header = [th.text.strip(u'\ue002') for th in pheader]
        header = ['FN', 'LN'] + header[3:-1]
        data = []
        for i in range(pages[pos]):
            driver.get(url.format(i=25 * i))
            time.sleep(3)
            elem = driver.find_element_by_class_name("Table")
            soup = BeautifulSoup(elem.get_attribute("innerHTML"), "html.parser")
            pdata = [[td.text for td in row.find_all('td')] for row in soup.find_all('tr') if len(row.find_all('td')) == len(pheader)]
            data += [row[1].split('\n')[2].split(' ')[:2] + row[3:-1] for row in pdata]
    except AttributeError:
        print("Failed: " + pos + " " + str(week))
        filename = 'yahoo_{pos}_week{week}.csv'.format(pos=pos, week=week)
        return
    filename = 'yahoo_{pos}_week{week}.csv'.format(pos=pos, week=week)
    with open(filename, 'w') as f:
        print('writing: ', filename)
        writer = csv.writer(f)
        try:
        #header = [h.strip(u'\ue002') for h in header]
        #data = [d.strip(u'\ue002') for ds in data for d in ds]
            writer.writerow(header)
            writer.writerows(data)
        except UnicodeEncodeError:
            print("Failed: " + pos + " " + str(week))
            print(header)
            print(data)
            return

In [41]:
url = 'https://football.fantasysports.yahoo.com/f1/731507/players?status=A&pos=O&cut_type=9&stat1=S_PW_12&myteam=0&sort=AR&sdir=1&count=0'

In [42]:
driver = webdriver.Firefox()
driver.get(url)

In [43]:
elem = driver.find_element_by_id("login-username")
elem.clear()
elem.send_keys("weinky")
elem.send_keys(Keys.RETURN)
time.sleep(2)
# Manually enter the password in the password box and hit return
# or fill the password in for "" here and run this cell
elem = driver.find_element_by_id("login-passwd")
elem.clear()
elem.send_keys("sinner2k")
elem.send_keys(Keys.RETURN)

In [16]:
os.chdir('yahoo')

WindowsError: [Error 2] The system cannot find the file specified: 'yahoo'

In [60]:
[cbs(pos, week) for pos in pages for week in weeks]

('writing: ', 'yahoo_QB_week1.csv')
('writing: ', 'yahoo_QB_week2.csv')
('writing: ', 'yahoo_QB_week3.csv')
('writing: ', 'yahoo_QB_week4.csv')
('writing: ', 'yahoo_QB_week5.csv')
('writing: ', 'yahoo_QB_week6.csv')
('writing: ', 'yahoo_QB_week7.csv')
('writing: ', 'yahoo_QB_week8.csv')
('writing: ', 'yahoo_QB_week9.csv')
('writing: ', 'yahoo_QB_week10.csv')
('writing: ', 'yahoo_QB_week11.csv')
('writing: ', 'yahoo_QB_week12.csv')
('writing: ', 'yahoo_QB_week13.csv')
('writing: ', 'yahoo_QB_week14.csv')
('writing: ', 'yahoo_QB_week15.csv')
('writing: ', 'yahoo_QB_week16.csv')
('writing: ', 'yahoo_QB_week17.csv')
('writing: ', 'yahoo_WR_week1.csv')
('writing: ', 'yahoo_WR_week2.csv')
('writing: ', 'yahoo_WR_week3.csv')
('writing: ', 'yahoo_WR_week4.csv')
('writing: ', 'yahoo_WR_week5.csv')
('writing: ', 'yahoo_WR_week6.csv')
('writing: ', 'yahoo_WR_week7.csv')
('writing: ', 'yahoo_WR_week8.csv')
('writing: ', 'yahoo_WR_week9.csv')
('writing: ', 'yahoo_WR_week10.csv')
('writing: ', 'yaho

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [52]:
[cbs(pos, week) for pos in pages for week in [17]]

Failed: QB 17
Failed: WR 17
Failed: RB 17
Failed: TE 17


[None, None, None, None]

In [57]:
os.chdir('..')

In [58]:
cbs('QB', 1)

('writing: ', 'yahoo_QB_week1.csv')
